In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.optim import AdamW
from transformers import RobertaForSequenceClassification
from sklearn.metrics import f1_score
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

import torch
from torch.optim import AdamW
from transformers import RobertaForSequenceClassification
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 
import os 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import torch

d:\Proyectos\TESIS 2024\depression_detection\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.is_available()

False

In [4]:
#Se leen los datos y se seleccionan las variables que presentan una diferencia
# entre los individuos que presentan estres y los que no
data_path = Path(os.getcwd()).parent / "data"
silver_path = data_path / "silver" / "dreadditCleanTrain.csv"
df = pd.read_csv(silver_path, usecols= ["text","clean_text","clean_text_sentence_sep",
                                        "singular_pronouns", "avg_word_len",'lex_diversity',"label"])

In [5]:

model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Ejemplo de cómo usar el dataset
dataset = SentimentDataset(df['clean_text'].tolist(), df['label'].tolist(), tokenizer, max_length=128)
loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [7]:
torch.cuda.is_available()

True

In [8]:
# Suponiendo que 'model' y 'loader' están definidos y configurados correctamente
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

# Listas para almacenar las métricas
losses = []
f1_scores = []

In [10]:

def train_and_evaluate(model, train_loader, val_loader, device, num_epochs=5):
    optimizer = AdamW(model.parameters(), lr=1e-5)
    best_f1 = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)

        # Evaluación
        model.eval()
        val_true, val_preds = [], []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)

                val_true.extend(labels.cpu().numpy())
                val_preds.extend(predictions.cpu().numpy())

        val_f1 = f1_score(val_true, val_preds, average='binary')
        if val_f1 > best_f1:
            best_f1 = val_f1  # Actualizar el mejor F1 score visto hasta ahora

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation F1: {val_f1:.4f}')

    return best_f1


In [11]:
from torch.utils.data import Subset, DataLoader

# Inicializar el KFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Convertir labels a numpy para el uso con StratifiedKFold
labels = np.array(df['label'].tolist())

# Ejecutar la validación cruzada
f1_scores = []
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    train_subsampler = Subset(dataset, train_idx)
    val_subsampler = Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_subsampler, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_subsampler, batch_size=16, shuffle=False)

    print(f"Training fold {fold+1}")
    fold_f1 = train_and_evaluate(model, train_loader, val_loader, device)
    f1_scores.append(fold_f1)

print(f"Average F1 across folds: {np.mean(f1_scores):.4f}")

Training fold 1
Epoch 1/3, Train Loss: 0.6318, Validation F1: 0.8197
Epoch 2/3, Train Loss: 0.3882, Validation F1: 0.8008
Epoch 3/3, Train Loss: 0.3020, Validation F1: 0.8430
Training fold 2
Epoch 1/3, Train Loss: 0.3104, Validation F1: 0.9565
Epoch 2/3, Train Loss: 0.2282, Validation F1: 0.9270
Epoch 3/3, Train Loss: 0.1274, Validation F1: 0.9327
Training fold 3
Epoch 1/3, Train Loss: 0.1697, Validation F1: 0.9946
Epoch 2/3, Train Loss: 0.0892, Validation F1: 0.9788
Epoch 3/3, Train Loss: 0.0570, Validation F1: 0.9661
Average F1 across folds: 0.9314


In [12]:
def save_model_and_tokenizer(model, tokenizer, save_directory):
    """
    Save the fine-tuned model and tokenizer to a specified directory.

    Args:
        model: The trained RoBERTa model (e.g., an instance of RobertaForSequenceClassification).
        tokenizer: The tokenizer used with the RoBERTa model.
        save_directory (str): The path to the directory where the model and tokenizer will be saved.

    Returns:
        None
    """
    # Create the directory if it does not exist
    import os
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Save the model
    model.save_pretrained(save_directory)
    
    # Save the tokenizer associated with the model
    tokenizer.save_pretrained(save_directory)

    print(f"Model and tokenizer have been saved to {save_directory}") 

In [21]:
ruta = Path(os.getcwd()).parent / "models"

In [17]:
#save_model_and_tokenizer(model, tokenizer, ruta)

Model and tokenizer have been saved to c:\Users\DELLPHOTO\depression_detection\models


# Inferencia

In [19]:
def prepare_text_for_inference(text, tokenizer, max_length=128):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,  # Agrega los tokens especiales para RoBERTa '[CLS]' y '[SEP]'
        max_length=max_length,    # Trunca o rellena el texto hasta la longitud máxima
        padding='max_length',     # Rellena hasta `max_length`
        truncation=True,          # Trunca a `max_length` si el texto es más largo
        return_attention_mask=True,
        return_tensors='pt',      # Retorna tensores de PyTorch
    )
    return encoding['input_ids'], encoding['attention_mask']

def make_prediction(text, model, tokenizer, device):
    model.eval()  # Pone el modelo en modo evaluación

    input_ids, attention_mask = prepare_text_for_inference(text, tokenizer)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():  # Desactiva el cálculo de gradientes para inferencia
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1)

    return probabilities.cpu().numpy(), predicted_class.cpu().numpy()


# Ejemplo de uso
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = ruta
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.to(device)  # Asegúrate de que el modelo esté en el mismo dispositivo (CPU o GPU)

text = "Yesterday i went to the movies with my friends"  # Texto de ejemplo para clasificación
probabilities, predicted_class = make_prediction(text, model, tokenizer, device)
print(f"Probabilidades: {probabilities}")
print(f"Clase Predicha: {predicted_class}")



Probabilidades: [[9.9740702e-01 2.3933384e-03 1.9965124e-04]]
Clase Predicha: [0]
